# Healthcare spending, part 2

In [1]:
#install.packages("lfe")
#install.packages("stargazer")
suppressWarnings(library(lfe))
suppressWarnings(library(stargazer))
suppressWarnings(library(dplyr))

Loading required package: Matrix

Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



## 1. import data and cleanup

In [2]:
getwd()
sample <- read.csv("data/health-spending.csv", nrow=10)
head(sample)
sapply(sample, class)
dim(sample)

[1] "C:/Users/wue04/Box Sync/healthcare-spending-fiscal-rules/part2-cost-shifting"

country,year,bailout,gdp,pop,debt,POP_80.,GOV_LEFT,FR,er_level,bbr_nat_2lvls,OOP_Exp,CURE,CARE,PREV,PHARM,BEDS,BEDS_LTC,CAP.FORMATION,HOSPITAL_EXP
Australia,1985,0,4.03e+11,15788310,NA,300.0,100,1,1,0,7255.928,14824.11,NA,254.089,1157.520,98534,NA,2432.782,9455.085
Australia,1986,0,4.11e+11,16018350,NA,314.3,100,1,1,0,8023.363,15381.10,NA,284.003,1296.217,NA,NA,2580.394,9785.289
Australia,1987,0,4.33e+11,16263870,NA,327.6,100,1,1,0,8325.094,15663.99,NA,320.267,1476.931,94653,NA,2718.716,10086.669
Australia,1988,0,4.53e+11,16532160,NA,340.5,100,1,1,0,9123.358,15856.88,NA,89.551,1462.585,NA,NA,2649.690,10011.853
Australia,1989,0,4.72e+11,16814420,80455919201,354.1,100,0,0,0,9763.080,16179.57,NA,118.617,1580.287,91312,NA,2663.733,9909.812
Australia,1990,0,4.78e+11,17065130,78639034361,367.3,100,0,0,0,10681.239,16821.24,NA,4.270,1511.820,NA,NA,2573.098,10036.322


country          year       bailout           gdp           pop 
     "factor"     "integer"     "integer"     "numeric"     "integer" 
         debt       POP_80.      GOV_LEFT            FR      er_level 
    "numeric"     "numeric"     "integer"     "integer"     "integer" 
bbr_nat_2lvls       OOP_Exp          CURE          CARE          PREV 
    "integer"     "numeric"     "numeric"     "logical"     "numeric" 
        PHARM          BEDS      BEDS_LTC CAP.FORMATION  HOSPITAL_EXP 
    "numeric"     "integer"     "logical"     "numeric"     "numeric"

[1] 10 20

In [2]:
health <- read.csv("data/health-spending.csv",
                  col.names=c("country", "year", "bailout", "gdp", "pop", "debt", "pop80",
                             "gov_left", "fr", "er", "bbr", "oop", "cure",
                             "care", "prev", "pharm", "beds", "beds_ltc", "cap", "hospital_exp", "rr"))
head(health)
health$country <- as.character(health$country)
sapply(health, class)

country,year,bailout,gdp,pop,debt,pop80,gov_left,fr,er,...,oop,cure,care,prev,pharm,beds,beds_ltc,cap,hospital_exp,rr
Australia,1985,0,4.03e+11,15788310,NA,300.0,100,1,1,...,7255.928,14824.11,NA,254.089,1157.520,98534,NA,2432.782,9455.085,1
Australia,1986,0,4.11e+11,16018350,NA,314.3,100,1,1,...,8023.363,15381.10,NA,284.003,1296.217,NA,NA,2580.394,9785.289,1
Australia,1987,0,4.33e+11,16263870,NA,327.6,100,1,1,...,8325.094,15663.99,NA,320.267,1476.931,94653,NA,2718.716,10086.669,1
Australia,1988,0,4.53e+11,16532160,NA,340.5,100,1,1,...,9123.358,15856.88,NA,89.551,1462.585,NA,NA,2649.690,10011.853,1
Australia,1989,0,4.72e+11,16814420,80455919201,354.1,100,0,0,...,9763.080,16179.57,NA,118.617,1580.287,91312,NA,2663.733,9909.812,0
Australia,1990,0,4.78e+11,17065130,78639034361,367.3,100,0,0,...,10681.239,16821.24,NA,4.270,1511.820,NA,NA,2573.098,10036.322,0


country         year      bailout          gdp          pop         debt 
 "character"    "integer"    "integer"    "numeric"    "numeric"    "numeric" 
       pop80     gov_left           fr           er          bbr          oop 
   "numeric"    "numeric"    "integer"    "integer"    "integer"    "numeric" 
        cure         care         prev        pharm         beds     beds_ltc 
   "numeric"    "numeric"    "numeric"    "numeric"    "numeric"    "numeric" 
         cap hospital_exp           rr 
   "numeric"    "numeric"    "integer"

In [3]:
# generate time lagged measuemrents
# t-1 and t-2
health <- health %>%
    group_by(country) %>%
    mutate(fr1 = lag(fr, 1))%>%
    mutate(fr2 = lag(fr, 2))%>%
    mutate(er1 = lag(er, 1))%>%
    mutate(er2 = lag(er, 2))%>%
    mutate(bbr1 = lag(bbr, 1))%>%
    mutate(bbr2 = lag(bbr, 2))%>%
    mutate(rr1 = lag(rr, 1))%>%
    mutate(rr2 = lag(rr, 2))

In [25]:
names(health)
head(health[, c("country", "bbr", "bbr1", "bbr2")])

[1] "country"      "year"         "bailout"      "gdp"          "pop"         
 [6] "debt"         "pop80"        "gov_left"     "fr"           "er"          
[11] "bbr"          "oop"          "cure"         "care"         "prev"        
[16] "pharm"        "beds"         "beds_ltc"     "cap"          "hospital_exp"
[21] "fr1"          "fr2"          "er1"          "er2"          "bbr1"        
[26] "bbr2"

country,bbr,bbr1,bbr2
Australia,0,NA,NA
Australia,0,0,NA
Australia,0,0,0
Australia,0,0,0
Australia,0,0,0
Australia,0,0,0


In [4]:
# data characteristics
table(health$er)
table(health$bbr)


  0   1 
748 212 


  0   1 
689 271 

In [ ]:
rm(sample)

## 2. regressions
### budget shifting
log(out_of_pocket_exp) = FR/ER/BBR + log(gdp) + log(debt) + log(pop) + bailout + fe(year) + fe(country)

In [5]:
shift_fr <- felm(log(oop)~fr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

shift_er <- felm(log(oop)~er+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

shift_bbr <- felm(log(oop)~bbr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

shift_rr <- felm(log(oop)~rr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

shift_all <- felm(log(oop)~er+bbr+rr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

stargazer(shift_fr, shift_er, shift_bbr, shift_rr, shift_all, type="html",
          title="Table 1 - Budget Shifting",
          dep.var.labels="log(out of pocket expense)",
          covariate.labels=c("FR", "ER", "BBR", "RR", "log(Debt)", "log(GDP)", 
                             "log(Population)", "IMF Bailout"),
          out="output/table1-budget_shift.html")


<table style="text-align:center"><caption><strong>Table 1 - Budget Shifting</strong></caption>
<tr><td colspan="6" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="5"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="5" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="5">log(out of pocket expense)</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td></tr>
<tr><td colspan="6" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">FR</td><td>0.095<sup>***</sup></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.028)</td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left">ER</td><td></td><td>-0.113<sup>***</sup></td><td></td><td></td><td>-0.090<sup>

### budget setting
log(cure) = FR/ER/BBR + log(gdp) + log(debt) + log(pop) + bailout  + fe(year) + fe(country)

In [7]:
cure_fr <- felm(log(cure)~fr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

cure_er <- felm(log(cure)~er+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

cure_bbr <- felm(log(cure)~bbr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

cure_rr <- felm(log(cure)~rr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

cure_all <- felm(log(cure)~er+bbr+rr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

stargazer(cure_fr, cure_er, cure_bbr, cure_rr, cure_all, type="html", 
          title="Table 2 - Budget Setting - Curative Care",
          dep.var.labels="log(Curative care)",
          covariate.labels=c("FR", "ER", "BBR", "RR", "log(Debt)", "log(GDP)",
                            "log(Population)", "IMF bailout"),
          out="output/table2-curative.html")


<table style="text-align:center"><caption><strong>Table 2 - Budget Setting - Curative Care</strong></caption>
<tr><td colspan="6" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="5"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="5" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="5">log(Curative care)</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td></tr>
<tr><td colspan="6" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">FR</td><td>0.028<sup>*</sup></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.015)</td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left">ER</td><td></td><td>-0.004</td><td></td><td></td><td>-0.011</td></tr>
<tr

log(care) = FR/ER/BBR + log(gdp) + log(debt) + log(pop80) + bailout + fe(year) + fe(country)

In [8]:
care_fr <- felm(log(care)~fr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)

care_er <- felm(log(care)~er+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)

care_bbr <- felm(log(care)~bbr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)

care_rr <- felm(log(care)~rr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)

care_all <- felm(log(care)~er+bbr+rr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)

stargazer(care_fr, care_er, care_bbr, care_rr, care_all, type="html",
          title="Table 3 - Budget Setting - Long Term Care",
          dep.var.labels="log(long term care)",
          covariate.labels=c("FR", "ER", "BBR", "RR", "log(Debt)", "log(GDP)", 
                             "log(Population 80 above)", "IMF Bailout"),
          out="output/table3-longterm.html")


<table style="text-align:center"><caption><strong>Table 3 - Budget Setting - Long Term Care</strong></caption>
<tr><td colspan="6" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="5"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="5" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="5">log(long term care)</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td></tr>
<tr><td colspan="6" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">FR</td><td>-0.060</td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.088)</td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left">ER</td><td></td><td>0.149<sup>**</sup></td><td></td><td></td><td>0.155<sup>**</sup

log(prev) = FR/ER/BBR + log(gdp) + log(debt) + log(pop) + bailout + gov_left + fe(year) + fe(country)

In [9]:
prev_fr <- felm(log(prev)~fr+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)

prev_er <- felm(log(prev)~er+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)

prev_bbr <- felm(log(prev)~bbr+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)

prev_rr <- felm(log(prev)~rr+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)

prev_all <- felm(log(prev)~er+bbr+rr+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)

stargazer(prev_fr, prev_er, prev_bbr, prev_all, type="html", 
          title="Table 4 - Budget Setting - Preventative Care",
          dep.var.labels="log(Preventative care)",
          covariate.labels=c("FR", "ER", "BBR", "RR", "log(Debt)", "log(GDP)",
                            "log(Population)", "IMF bailout", "Govt lean left"),
          out="output/table4-preventative.html")


<table style="text-align:center"><caption><strong>Table 4 - Budget Setting - Preventative Care</strong></caption>
<tr><td colspan="5" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="4"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="4" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="4">log(Preventative care)</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td></tr>
<tr><td colspan="5" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">FR</td><td>0.317<sup>***</sup></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.116)</td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left">ER</td><td></td><td>-0.263<sup>***</sup></td><td></td><td>-0.285<sup>***</sup></td></tr>
<tr><td style

log(pharm) = FR/ER/BBR + log(gdp) + log(debt) + log(pop) + bailout + fe(year) + fe(country)

In [10]:
pharm_fr <- felm(log(pharm)~fr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

pharm_er <- felm(log(pharm)~er+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

pharm_bbr <- felm(log(pharm)~bbr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

pharm_rr <- felm(log(pharm)~rr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

pharm_all <- felm(log(pharm)~er+bbr+rr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

stargazer(pharm_fr, pharm_er, pharm_bbr, pharm_rr, pharm_all, type="html",
          title="Table 5 - Budget Setting - Pharmaceutical Spending",
          dep.var.labels="log(pharmaceutical spending)",
          covariate.labels=c("FR", "ER", "BBR", "RR", "log(Debt)", "log(GDP)", 
                             "log(Population)", "IMF Bailout"),
          out="output/table5-pharm.html")


<table style="text-align:center"><caption><strong>Table 5 - Budget Setting - Pharmaceutical Spending</strong></caption>
<tr><td colspan="6" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="5"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="5" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="5">log(pharmaceutical spending)</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td></tr>
<tr><td colspan="6" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">FR</td><td>-0.015</td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.038)</td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left">ER</td><td></td><td>0.024</td><td></td><td></td><td>0.010</td></

### direct and indirect control of supply
log(beds) = FR/ER/BBR + log(gdp) + log(debt) + log(pop80) + bailout + fe(year) + fe(country)

In [11]:
beds_fr <- felm(log(beds)~fr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)

beds_er <- felm(log(beds)~er+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)

beds_bbr <- felm(log(beds)~bbr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)

beds_rr <- felm(log(beds)~rr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)

beds_all <- felm(log(beds)~er+bbr+rr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)

stargazer(beds_fr, beds_er, beds_bbr, beds_rr, beds_all, type="html",
          title="Table 6 - Supply Control - Hospital Beds",
          dep.var.labels="log(hospital beds)",
          covariate.labels=c("FR", "ER", "BBR", "RR", "log(Debt)", "log(GDP)", 
                             "log(Population 80 above)", "IMF Bailout"),
          out="output/table6-beds.html")


<table style="text-align:center"><caption><strong>Table 6 - Supply Control - Hospital Beds</strong></caption>
<tr><td colspan="6" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="5"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="5" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="5">log(hospital beds)</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td></tr>
<tr><td colspan="6" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">FR</td><td>0.075<sup>***</sup></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.015)</td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left">ER</td><td></td><td>0.009</td><td></td><td></td><td>0.003</td></tr>
<tr

log(beds_ltc) = FR/ER/BBR + log(gdp) + log(debt) + log(pop80) + bailout + fe(year) + fe(country)

In [12]:
bedltc_fr <- felm(log(beds_ltc)~fr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
summary(bedltc_fr)

bedltc_er <- felm(log(beds_ltc)~er+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
summary(bedltc_er)

bedltc_bbr <- felm(log(beds_ltc)~bbr+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
summary(bedltc_bbr)

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
"the matrix is either rank-deficient or indefinite"

Error in chol.default(mat[ok, ok]) : 'a' must have dims > 0


Warning message in summary.felm(bedltc_fr):
"can't compute cluster F-test"ERROR while rich displaying an object: Error in quantile.default(x$residuals): missing values and NaN's not allowed if 'na.rm' is FALSE

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_conte

Error in chol.default(mat[ok, ok]) : 'a' must have dims > 0


Warning message in summary.felm(bedltc_er):
"can't compute cluster F-test"ERROR while rich displaying an object: Error in quantile.default(x$residuals): missing values and NaN's not allowed if 'na.rm' is FALSE

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_conte

Error in chol.default(mat[ok, ok]) : 'a' must have dims > 0


Warning message in summary.felm(bedltc_bbr):
"can't compute cluster F-test"ERROR while rich displaying an object: Error in quantile.default(x$residuals): missing values and NaN's not allowed if 'na.rm' is FALSE

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_cont

In [83]:
class(health$beds_ltc)
summary(health$beds_ltc)
length(health$country)

[1] "numeric"

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   1032   25653   67888  199749  165741 1946035     227 

[1] 960

log(capital) = FR/ER/BBR + log(gdp) + log(debt) + log(pop) + bailout + fe(year) + fe(country)

In [12]:
cap_fr <- felm(log(cap)~fr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

cap_er <- felm(log(cap)~er+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

cap_bbr <- felm(log(cap)~bbr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

cap_rr <- felm(log(cap)~rr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

cap_all <- felm(log(cap)~er+bbr+rr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

stargazer(cap_fr, cap_er, cap_bbr, cap_all, type="html",
          title="Table 8 - Supply Control - Capital Investment",
          dep.var.labels="log(capital investment)",
          covariate.labels=c("FR", "ER", "BBR", "RR", "log(Debt)", "log(GDP)", 
                             "log(Population)", "IMF Bailout"),
          out="output/table8-capital.html")


<table style="text-align:center"><caption><strong>Table 8 - Supply Control - Capital Investment</strong></caption>
<tr><td colspan="5" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="4"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="4" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="4">log(capital investment)</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td></tr>
<tr><td colspan="5" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">FR</td><td>0.001</td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.060)</td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left">ER</td><td></td><td>-0.040</td><td></td><td>-0.082</td></tr>
<tr><td style="text-align:left"></td><td></td><td>(0.

In [14]:
ls()

[1] "beds_all"  "beds_bbr"  "beds_er"   "beds_fr"   "beds_rr"   "cap_all"  
 [7] "cap_bbr"   "cap_er"    "cap_fr"    "cap_rr"    "care_all"  "care_bbr" 
[13] "care_er"   "care_fr"   "care_rr"   "cure_all"  "cure_bbr"  "cure_er"  
[19] "cure_fr"   "cure_rr"   "health"    "pharm_all" "pharm_bbr" "pharm_er" 
[25] "pharm_fr"  "pharm_rr"  "prev_all"  "prev_bbr"  "prev_er"   "prev_fr"  
[31] "prev_rr"   "shift_all" "shift_bbr" "shift_er"  "shift_fr"  "shift_rr"

### robustness check

Use hospital expenditure instead of curative care as DV.

log(hospital_exp) = FR/ER/BBR + log(gdp) + log(debt) + log(pop) + bailout  + fe(year) + fe(country)

In [15]:
hosexp_fr <- felm(log(hospital_exp)~fr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

hosexp_er <- felm(log(hospital_exp)~er+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

hosexp_bbr <- felm(log(hospital_exp)~bbr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

hosexp_rr <- felm(log(hospital_exp)~rr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

hosexp_all <- felm(log(hospital_exp)~er+bbr+rr+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

stargazer(hosexp_fr, hosexp_er, hosexp_bbr, hosexp_er, hosexp_all, type="html", 
          title="Table 9 - Budget Setting - Hospital Expenditure",
          dep.var.labels="log(Curative care)",
          covariate.labels=c("FR", "ER", "BBR", "RR", "log(Debt)", "log(GDP)",
                            "log(Population)", "IMF bailout"),
          out="output/table9-hospitalexp.html")


<table style="text-align:center"><caption><strong>Table 9 - Budget Setting - Hospital Expenditure</strong></caption>
<tr><td colspan="6" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="5"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="5" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="5">log(Curative care)</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td></tr>
<tr><td colspan="6" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">FR</td><td>-0.085<sup>***</sup></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td>(0.025)</td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left">ER</td><td></td><td>-0.008</td><td></td><td>-0.008</td><td>-0.0

Time lagged measurements: t-1 and t-2

In [39]:
shift_fr1 <- felm(log(oop)~fr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
cure_fr1 <- felm(log(cure)~fr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
care_fr1 <- felm(log(care)~fr1+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
prev_fr1 <- felm(log(prev)~fr1+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)
pharm_fr1 <- felm(log(pharm)~fr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
beds_fr1 <- felm(log(beds)~fr1+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
cap_fr1 <- felm(log(cap)~fr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
hosexp_fr1 <- felm(log(hospital_exp)~fr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

stargazer(shift_fr1, cure_fr1, care_fr1, prev_fr1, pharm_fr1, beds_fr1, cap_fr1, hosexp_fr1,
          type="html", 
          title="Table 10 - Time Lag 1 Year",
          dep.var.labels=c("Log(OOP)", "Log(Curative)", "Log(LTC)", "Log(Prev)", "Log(Pharm)", "Log(Beds)",
                           "Log(Capital)", "Log(Hospital Exp)"),
          covariate.labels=c("FR t-1", "log(Debt)", "log(GDP)", "log(Population)", "Log(Pop 80 above)",
                             "IMF bailout", "Left Gov"),
          out="output/table10-frlag1.html")


<table style="text-align:center"><caption><strong>Table 10 - Time Lag 1 Year</strong></caption>
<tr><td colspan="9" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="8"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="8" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>Log(OOP)</td><td>Log(Curative)</td><td>Log(LTC)</td><td>Log(Prev)</td><td>Log(Pharm)</td><td>Log(Beds)</td><td>Log(Capital)</td><td>Log(Hospital Exp)</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td><td>(7)</td><td>(8)</td></tr>
<tr><td colspan="9" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">FR t-1</td><td>0.070<sup>***</sup></td><td>0.007</td><td>-0.100</td><td>0.199<sup>*</sup></td><td>-0.040</td><td>0.059<sup>***</sup></td><td>-0.0005</td><td>-0.088<sup>***</sup></td></tr>
<tr><td style="text-align:left"></

In [38]:
shift_fr2 <- felm(log(oop)~fr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
cure_fr2 <- felm(log(cure)~fr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
care_fr2 <- felm(log(care)~fr2+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
prev_fr2 <- felm(log(prev)~fr2+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)
pharm_fr2 <- felm(log(pharm)~fr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
beds_fr2 <- felm(log(beds)~fr2+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
cap_fr2 <- felm(log(cap)~fr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
hosexp_fr2 <- felm(log(hospital_exp)~fr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

stargazer(shift_fr2, cure_fr2, care_fr2, prev_fr2, pharm_fr2, beds_fr2, cap_fr2, hosexp_fr2,
          type="html", 
          title="Table 11 - Time Lag 2 Years",
          dep.var.labels=c("Log(OOP)", "Log(Curative)", "Log(LTC)", "Log(Prev)", "Log(Pharm)", "Log(Beds)",
                           "Log(Capital)", "Log(Hospital Exp)"),
          covariate.labels=c("FR t-2", "log(Debt)", "log(GDP)", "log(Population)", "Log(Pop 80 above)",
                             "IMF bailout", "Left Gov"),
          out="output/table11-frlag2.html")


<table style="text-align:center"><caption><strong>Table 11 - Time Lag 2 Years</strong></caption>
<tr><td colspan="9" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="8"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="8" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>Log(OOP)</td><td>Log(Curative)</td><td>Log(LTC)</td><td>Log(Prev)</td><td>Log(Pharm)</td><td>Log(Beds)</td><td>Log(Capital)</td><td>Log(Hospital Exp)</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td><td>(7)</td><td>(8)</td></tr>
<tr><td colspan="9" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">FR t-2</td><td>0.048<sup>*</sup></td><td>-0.008</td><td>-0.136<sup>*</sup></td><td>0.055</td><td>-0.084<sup>**</sup></td><td>0.049<sup>***</sup></td><td>-0.008</td><td>-0.090<sup>***</sup></td></tr>
<tr><td style="text-al

In [22]:
shift_all1 <- felm(log(oop)~er1+bbr1+rr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
cure_all1 <- felm(log(cure)~er1+bbr1+rr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
care_all1 <- felm(log(care)~er1+bbr1+rr1+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
prev_all1 <- felm(log(prev)~er1+bbr1+rr1+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)
pharm_all1 <- felm(log(pharm)~er1+bbr1+rr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
beds_all1 <- felm(log(beds)~er1+bbr1+rr1+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
cap_all1 <- felm(log(cap)~er1+bbr1+rr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
hosexp_all1 <- felm(log(hospital_exp)~er1+bbr1+rr1+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

stargazer(shift_all1, cure_all1, care_all1, prev_all1, pharm_all1, beds_all1, cap_all1, hosexp_all1,
          type="html", 
          title="Table 12 - ER+BBR - Time Lag 1 Year",
          dep.var.labels=c("Log(OOP)", "Log(Curative)", "Log(LTC)", "Log(Prev)", "Log(Pharm)", "Log(Beds)",
                           "Log(Capital)", "Log(Hospital Exp)"),
          covariate.labels=c("ER t-1", "BBR t-1", "RR t-1", "log(Debt)", "log(GDP)", "log(Population)",
                             "Log(Pop 80 above)", "IMF bailout", "Left Gov"),
          out="output/table12-alllag1.html")


<table style="text-align:center"><caption><strong>Table 12 - ER+BBR - Time Lag 1 Year</strong></caption>
<tr><td colspan="9" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="8"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="8" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>Log(OOP)</td><td>Log(Curative)</td><td>Log(LTC)</td><td>Log(Prev)</td><td>Log(Pharm)</td><td>Log(Beds)</td><td>Log(Capital)</td><td>Log(Hospital Exp)</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td><td>(7)</td><td>(8)</td></tr>
<tr><td colspan="9" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">ER t-1</td><td>-0.071<sup>***</sup></td><td>-0.033<sup>**</sup></td><td>0.170<sup>**</sup></td><td>-0.278<sup>***</sup></td><td>-0.008</td><td>-0.016</td><td>-0.030</td><td>-0.031</td></tr>
<tr><td style="text-ali

In [23]:
shift_all2 <- felm(log(oop)~er2+bbr2+rr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
cure_all2 <- felm(log(cure)~er2+bbr2+rr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
care_all2 <- felm(log(care)~er2+bbr2+rr2+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
prev_all2 <- felm(log(prev)~er2+bbr2+rr2+log(debt)+log(gdp)+log(pop)+bailout+gov_left|as.factor(year)+as.factor(country),
             data=health)
pharm_all2 <- felm(log(pharm)~er2+bbr2+rr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
beds_all2 <- felm(log(beds)~er2+bbr2+rr2+log(debt)+log(gdp)+log(pop80)+bailout|as.factor(year)+as.factor(country),
             data=health)
cap_all2 <- felm(log(cap)~er2+bbr2+rr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)
hosexp_all2 <- felm(log(hospital_exp)~er2+bbr2+rr2+log(debt)+log(gdp)+log(pop)+bailout|as.factor(year)+as.factor(country),
             data=health)

stargazer(shift_all2, cure_all2, care_all2, prev_all2, pharm_all2, beds_all2, cap_all2, hosexp_all2,
          type="html", 
          title="Table 13 - ER+BBR - Time Lag 2 Years",
          dep.var.labels=c("Log(OOP)", "Log(Curative)", "Log(LTC)", "Log(Prev)", "Log(Pharm)", "Log(Beds)",
                           "Log(Capital)", "Log(Hospital Exp)"),
          covariate.labels=c("ER t-2", "BBR t-2", "RR t-2", "log(Debt)", "log(GDP)", "log(Population)", "Log(Pop 80 above)",
                             "IMF bailout", "Left Gov"),
          out="output/table13-alllag2.html")


<table style="text-align:center"><caption><strong>Table 13 - ER+BBR - Time Lag 2 Years</strong></caption>
<tr><td colspan="9" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="8"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="8" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>Log(OOP)</td><td>Log(Curative)</td><td>Log(LTC)</td><td>Log(Prev)</td><td>Log(Pharm)</td><td>Log(Beds)</td><td>Log(Capital)</td><td>Log(Hospital Exp)</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td><td>(4)</td><td>(5)</td><td>(6)</td><td>(7)</td><td>(8)</td></tr>
<tr><td colspan="9" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">ER t-2</td><td>-0.054<sup>**</sup></td><td>-0.044<sup>***</sup></td><td>0.114</td><td>-0.359<sup>***</sup></td><td>-0.037</td><td>-0.029<sup>*</sup></td><td>-0.0003</td><td>-0.055<sup>**</sup></td></tr>
<tr><td s

In [18]:
ls()

[1] "beds_all"    "beds_all1"   "beds_all2"   "beds_bbr"    "beds_er"    
 [6] "beds_fr"     "beds_rr"     "cap_all"     "cap_all1"    "cap_all2"   
[11] "cap_bbr"     "cap_er"      "cap_fr"      "cap_rr"      "care_all"   
[16] "care_all1"   "care_all2"   "care_bbr"    "care_er"     "care_fr"    
[21] "care_rr"     "cure_all"    "cure_all1"   "cure_all2"   "cure_bbr"   
[26] "cure_er"     "cure_fr"     "cure_rr"     "health"      "hosexp_all" 
[31] "hosexp_all1" "hosexp_all2" "hosexp_bbr"  "hosexp_er"   "hosexp_fr"  
[36] "hosexp_rr"   "pharm_all"   "pharm_all1"  "pharm_all2"  "pharm_bbr"  
[41] "pharm_er"    "pharm_fr"    "pharm_rr"    "prev_all"    "prev_all1"  
[46] "prev_all2"   "prev_bbr"    "prev_er"     "prev_fr"     "prev_rr"    
[51] "shift_all"   "shift_all1"  "shift_all2"  "shift_bbr"   "shift_er"   
[56] "shift_fr"    "shift_rr"

In [24]:
rm(list=ls(pattern="\\_"))